# SHAP 101 - explaining ml models and beyond

### Feature Attributions
* SHAP (SHapley Additive exPlanations) - https://shap.readthedocs.io/en/latest/
* Understand individual predictions - https://www.kaggle.com/code/dansbecker/shap-values/tutorial
* Aggregate SHAP values for even more detailed model insights - https://www.kaggle.com/code/dansbecker/advanced-uses-of-shap-values/tutorial
* Style SHAP Plots - https://towardsdatascience.com/how-to-easily-customize-shap-plots-in-python-fdff9c0483f2?gi=754319eae2c8

* Convert SHAP Score to percentage - https://medium.com/towards-data-science/* black-box-models-are-actually-more-explainable-than-a-logistic-regression-f263c22795d
* base value in SHAP (average of the outcome variable in the training set) - https://towardsdatascience.com/explainable-ai-xai-with-shap-multi-class-classification-problem-64dd30f97cea
* Additive Feature importance - https://medium.com/@singhvis929/additive-feature-attribution-methods-diving-into-ml-explainability-98c81c656d3d
* Kernel in SHAP - https://towardsdatascience.com/one-feature-attribution-method-to-supposedly-rule-them-all-shapley-values-f3e04534983d

### Partial Dependence Plot
* Partial Dependence Plot Theory - https://christophm.github.io/interpretable-ml-book/pdp.html
* Partial Dependence Plots - https://scikit-learn.org/stable/modules/partial_dependence.html
* Partial Dependence Plots show how a feature affects predictions - https://www.kaggle.com/code/dansbecker/partial-plots

### Additional References
* Fairlearn - https://fairlearn.org
* squaredev.io - https://github.com/squaredev-io/explainable-ai
* Rerun-sdk - https://pypi.org/project/rerun-sdk/

### Pandas 
* Working with missing value - https://pandas.pydata.org/docs/user_guide/missing_data.html
* show max columns - https://stackoverflow.com/questions/11707586/how-do-i-expand-the-output-display-to-see-more-columns-of-a-pandas-dataframe/11711637#11711637
* DataFrame types with dtypes - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html

### Scikit-Learn
* Custom Demo Classifier _ https://scikit-learn.org/stable/developers/develop.html
* Custom Classification Model - https://towardsdatascience.com/how-to-create-custom-scikit-learn-classification-and-regression-models-70db7e76addd
* Custom ensemble Model - https://towardsdatascience.com/how-to-build-a-custom-estimator-for-scikit-learn-fddc0cb9e16e?gi=b6f190dcd368
* Custom Regression Model - https://towardsdatascience.com/building-a-custom-model-in-scikit-learn-b0da965a1299

### Model validation
Confusion Matrix, ROC, AUC - https://towardsdatascience.com/intuition-behind-roc-auc-score-1456439d1f30?gi=c4b96aa0c60e

### Explainable AI
* The Challenge of Crafting Intelligible Intelligence - https://cacm.acm.org/magazines/2019/6/237004-the-challenge-of-crafting-intelligible-intelligence/fulltext

# Kaggle Titanic Compitition
https://www.kaggle.com/competitions/titanic

### Data Description
https://www.kaggle.com/competitions/titanic/data?select=train.csv

| Variable | Definition	| Key | 
| :--- | :--- | :--- |
| survival | Survival |	0 = No, 1 = Yes |
| pclass   | Ticket class |	1 = 1st, 2 = 2nd, 3 = 3rd |
| sex |	Sex	| |
| Age |	Age | in years | 	
| sibsp	| # of siblings / spouses <br>aboard the Titanic |	
| parch	| # of parents / children <br>aboard the Titanic |	
| ticket |	Ticket number | |	
| fare | Passenger fare | (Ticket price paid)  |	
| cabin	| Cabin number | |	
| embarked	| Port of Embarkation |	C = Cherbourg, <br>Q = Queenstown, <br>S = Southampton |

### Variable Notes

**pclass:** A proxy for socio-economic status (SES)
* 1st = Upper
* 2nd = Middle
* 3rd = Lower

**age:** Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

**sibsp:** The dataset defines family relations in this way...
* Sibling = brother, sister, stepbrother, stepsister
* Spouse = husband, wife (mistresses and fiancés were ignored)

**parch:** The dataset defines family relations in this way...
* Parent = mother, father
* Child = daughter, son, stepdaughter, stepson

Some children travelled only with a nanny, therefore parch=0 for them.

In [ ]:
# %load_ext autoreload
# %autoreload 2

# load JS visualization code to notebook
import shap
# used for render kernelshap explainer inference progressbar 
shap.initjs()

# graphic display modes
%matplotlib inline

# DARK_MODE = True
DARK_MODE = False

"""the custom argo presented in this notebook is not suitable for kernel explainer"""
# ALGO="custom"
# KERNEL_EXPLAINER = False

ALGO="xgboost"
KERNEL_EXPLAINER = True
# KERNEL_EXPLAINER = False

# LINK = "logit" # to transfer the shap value from probability space to log(odds) space to have value evenly distributed.
LINK = "identity" # to get the shap value of probability space

In [ ]:
from utils.datahelper import (
    KaggleData, 
    current_dir_subpath,
    profile, 
    feature_correlation, 
    na_columns,
    fill_missing_values_with_mean,
    DataVisualizer,
)

from utils.modelhelper import (
    ProbBinaryClassifier,
    ModelValidator,
    ModelExplainer,
    ModelKernelExplainer
)

from matplotlib import pyplot as plt
import seaborn as sns

import pandas as pd
from pandas import DataFrame
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

# set the pandas display
pd.set_option('display.max_columns', 15)
pd.set_option('display.width', 1000) # for print

# create a visualization helper object
visual_helper = DataVisualizer(dark_mode=DARK_MODE)

titanic_train_path = current_dir_subpath("data/train.csv")
titanic_test_path = current_dir_subpath("data/test.csv")
label_name = "Survived"
one_hot_cols = ["Sex"]

titanic = KaggleData(
     train_path = titanic_train_path,
     test_path = titanic_test_path,
     label_col=label_name
)

# load all raw unprocessed data as DataFrame, label as Series
# one_hot_cols transfers categorical column to one_hot encoded column 
train_X_raw_df, test_X_raw_df, train_raw_y = titanic.load(one_hot_cols=one_hot_cols)
all_X_raw_df = titanic.load_all(one_hot_cols=one_hot_cols)

## Explore numerical features in different data sets

In [ ]:
# detect the numerical features for building classifier
num_features = all_X_raw_df.describe().columns.to_list()
print(num_features)

In [ ]:
profile(train_X_raw_df, title="Profile of Raw Training Dataset")
print("\n" + "#" * 20)
profile(test_X_raw_df, title="Profile of Raw Test Dataset")

In [ ]:
# titanic.boxplot_dist(["PassengerId", "Fare", "Age"], marker="x", orient="h", legend="upper right", dark_mode=DARK_MODE)
titanic.boxplot_dist(["Fare", "Age"], marker="x", orient="h", legend="upper right", dark_mode=DARK_MODE)

In [ ]:
titanic.boxplot_dist(['Pclass', 'SibSp', 'Parch'], marker="x", orient="h", legend="upper right", dark_mode=DARK_MODE)

## Data Inputation

replace the NaN values of numerical features in training and test dataset

In [ ]:
# train_X_raw_df.describe()
# test_X_raw_df.describe()
# all_X_raw_df.describe()

In [ ]:
def filter_values(x):
    """x is tuple with two positions"""
    match x[0]:
        case "Age":
            return (x[0], round(x[1]))
        case "Fare":
            return (x[0], round(x[1], 4)) # returns float with rounded 4 decimals
        case _:
            return x   

In [ ]:
train_X_raw_df, train_mean_dict = fill_missing_values_with_mean(
    df=train_X_raw_df, pop_df=all_X_raw_df, filter_cols=num_features, filter_func=filter_values)
print(train_mean_dict)
print(f"no. of numerical cols. has NaN values: {len(na_columns(train_X_raw_df, num_features))}")

In [ ]:
test_X_raw_df, test_mean_dict = fill_missing_values_with_mean(
    df=test_X_raw_df, pop_df=all_X_raw_df, filter_cols=num_features, filter_func=filter_values)
print(test_mean_dict)
print(f"no. of numerical cols. has NaN values: {len(na_columns(test_X_raw_df, num_features))}")


## Examining the correlation of numerical features and labels

In [ ]:
train_X_y_raw_df = pd.concat([train_X_raw_df, train_raw_y], axis=1)
threshold = 0.2
corr_df, high_corr_df = feature_correlation(train_X_y_raw_df, label=label_name, threshold=threshold)
corr_df


In [ ]:
visual_helper.display_feature_correlation(corr_df=corr_df)

In [ ]:
# display the features with high correlation to the label
high_corr_df

## Select features

In [ ]:
# select only numerical features
selected_features = num_features.copy()
selected_features.remove("PassengerId")

In [ ]:
# split the training data
X_train, X_valid, y_train, y_valid = titanic.split(
    titanic.select_cols(df=train_X_raw_df, cols=selected_features), 
    train_raw_y, test_size=0.2, random_state=10)

## Algo factory for building different models

In [ ]:
from typing import Any
class AlgoML:
    """Interface"""
    def train_model(self, X_train, y_train) -> Any:
        pass

class AlgoXgboost(AlgoML):
    def train_model(self, X_train, y_train) -> Any:
        # shift + tab to unindent multiple lines 
        param_grid = {
            'n_estimators': range(6, 10),
            'max_depth' : range(3, 8),
            'learning_rate' : [.2, .3, .4],
            'colsample_bytree' : [.7, .8, .9, 1]
        }

        xgb = XGBClassifier()
        # Searching for the best parameters
        g_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, n_jobs=1, verbose=0, return_train_score=True)

        # Fitting the model using best parameters found
        g_search.fit(X_train, y_train)

        # print the best parameters found
        print(g_search.best_params_)
        # valid 
        # g_search.score(X_valid, y_valid)
        model = g_search
        return model
    
class AlgoCustom(AlgoML):
    def train_model(self, X_train, y_train) -> Any:
        feature_position = ProbBinaryClassifier.feature_position(X_train, "Sex_female")
        # config model
        model = ProbBinaryClassifier(feature_position, 1)
        # Train model
        model.fit(X_train, y_train)
        return model
    

class AlgoFactory:
    def get_algo(algo: str) -> AlgoML:
        if algo == "xgboost":
            return AlgoXgboost()
        else:
            return AlgoCustom()

## Traing a model 

* Xgboost model
* Custom model

In [ ]:
algo = AlgoFactory.get_algo(algo=ALGO)
model = algo.train_model(X_train=X_train, y_train=y_train)
predicts = model.predict(X_valid)

## Model performance validation

In [ ]:
validator = ModelValidator(y_valid, predicts, dark_mode=DARK_MODE)
scores_dict = validator.evaluate()
validator.print_eval_result(scores_dict)

In [ ]:
# 83 is false positive (false predicted to positive label, which has negative label as ground truth)
validator.display_confusion_matrix()

In [ ]:
# ROC: Receiver operating characteristic
# AUC -> 1
validator.display_roc_curve()

## Model input feature attribution

**local weighted regession surrogate explanation model**
Kernel SHAP is based on LIME (local Interpretable Model-agnostic Explanations), which uses its own kernel to create a weighted linear regression surrogate surrounding the local data. 

Note:
* The surrogate model is created for every instance x with sampled coalition. (https://christophm.github.io/interpretable-ml-book/shap.html), therefor the kernel SHAP is quite slow to execute.

<!--
 ![local weighted regession surrogate explanation model](https://dl.acm.org/cms/attachment/ 382dae73-8228-4137-9aae-889784b9d7c5/f5.jpg) 
-->
<img src="https://dl.acm.org/cms/attachment/382dae73-8228-4137-9aae-889784b9d7c5/f5.jpg" width="700" height="500">


In [ ]:
# Create Explainer callable for X_valid data set
if KERNEL_EXPLAINER:
    explainer = ModelKernelExplainer(model=model, train_data=X_train, inference_data=X_valid, dark_mode=DARK_MODE, link=LINK)
else: 
    explainer = ModelExplainer(model=model, data=X_valid, dark_mode=DARK_MODE)


In [ ]:
@ModelExplainer.valid_index
def get_details(df: DataFrame, idx: int, org_df: DataFrame) -> DataFrame:
    """get the passenger info from the original data frame based on the index position of validation set
    @param df: 
    @param idx: index position, this must be idx so that decorator works
    """
    # get the index name from the original raw dataset, from the index position of validation data set
    index_name = df.iloc[idx].name
    # use slicing on the same index name to get the passenger info as a DataFrame obj
    return org_df.loc[index_name: index_name]

## Base values in SHAP
Following the [SHAP paper](https://proceedings.neurips.cc/paper_files/paper/2017/file/8a20a8621978632d76c43dfd28b67767-Paper.pdf):

the base value $E[f(z)]$ is "the value that would be predicted if we did not know any features from the current output". 

In other words, it is the mean of model prediction using our `background` or `reference` dataset: `X_train`
```python3
ModelKernelExplainer(model=model, train_data=X_train, ...)
```
the base value $E[f(z)]$:
```python3
base_value = np.mean(
    model.predict_proba(X_train),
    axis=0
)
```

Note: 
* The kernelShap with `Link="identity"` output the shap value in `probability` unit space
* The kernelShap with `Link="logit"` output the shap value in `log(odd)` unit space, which is more evenly distributed for loss and gains in a linear scale.

Logit funciton $log_e(\frac{p_x}{1 - p_x}) = log_e(p_x) - log_e(1 - p_x)$, `np.log()` uses Euler's number $e = 2.71828$ as base, which is the natural logarithm.

Derivative of $e^x$ is $e^x$. 
In other words: $\frac{d}{dx}(e^x) = e^x$



In [ ]:
from typing import Iterable
def simple_logit(probs: Iterable):
    # assumption all the probability in the Iterable shall be 1
    return np.array([ np.log(x) - np.log(1-x) for x in probs])

if getattr(explainer.explainer, "expected_value", None) is not None:
    # print(explainer.explainer.expected_value)
    print(f"shap base value: \n{explainer.explainer.expected_value}")

    # calculate base value for kernelshap with link identity and logit output space mapper
    # average of all the local data predictions, X_train
    all_local_data_predictions = model.predict_proba(X_train)
    base_value = np.mean(all_local_data_predictions, axis=0)
    if (LINK == 'logit'):
        base_value = simple_logit(base_value)
    print(f"base value from local data predictions: \n{base_value}")
else:
    print("no expected_value in explainer")    

In [ ]:
# examining the shap_values, as Explaination or List, Array from kernelExplainer
# explainer.shap_values

In [ ]:
# shap.initjs()
# shap.plots.force(explainer.explainer.expected_value[1], 
#                  explainer.shap_values[1], explainer.data, link="identity")
# shap.plots.force(explainer.explainer.expected_value[1], explainer.shap_values[1][0], explainer.data.iloc[0], link="identity")
# shap.plots.force(explainer.explainer.expected_value[0], explainer.shap_values[0][0], explainer.data.iloc[0], link="identity")

In [ ]:
# Pclass = 3.0 doesn't play any rules for our passenger at X_valid dataset with rank pos 0
idx_pos = 0
passager_df = get_details(df=X_valid, idx=idx_pos, org_df = train_X_y_raw_df)
print(passager_df)
explainer.force_plot(idx = idx_pos)
explainer.waterfall_plot(idx = idx_pos)

In [ ]:
# Pclass = 3.0 doesn't play any rules for our passenger at X_valid dataset with rank pos 100
idx_pos = 100
passager_df = get_details(df=X_valid, idx=idx_pos, org_df = train_X_y_raw_df)
print(passager_df)
explainer.force_plot(idx = idx_pos)
explainer.waterfall_plot(idx = idx_pos)

In [ ]:
# idx_pos = 178
# passager_df = get_details(df=X_valid, idx=idx_pos, org_df = train_X_raw_df)
# print(passager_df)
# explainer.force_plot(idx = idx_pos)

In [ ]:
# summary_plot is a wrapper of beeswarm_plot(), kernel shap has no beeswarm_plot
explainer.beeswarm_plot()

In [ ]:
explainer.summary_plot()

In [ ]:
explainer.scatter_plot(feature="Age", auto_interact_features=1)

In [ ]:
# explainer.scatter_plot(feature="Age", interact_feature="Sex_female")

In [ ]:
explainer.scatter_plot(feature="Pclass")

In [ ]:
explainer.scatter_plot(feature="Fare", interact_feature="Pclass")

In [ ]:
explainer.scatter_plot(feature="Fare", interact_feature="Sex_female")

### Play ground: Naiv gender based approach

In [ ]:
"""Naiv gender based approach"""
X_y_train = pd.concat([X_train, y_train], axis=1)

# women = train_X_y_raw_df.loc[train_X_y_raw_df["Sex_female"] == 1]["Survived"]
# rate_women = sum(women) / len(women)

def survived_rate(df: DataFrame, feature_name="Sex_female", feature_value=1, label_name="Survived"):
    label_part = df.loc[df[feature_name] == feature_value][label_name]
    return sum(label_part) / len(label_part)

rate_women = survived_rate(X_y_train, "Sex_female")
print(f"% of women who survied: {rate_women:.2%}")

rate_men = survived_rate(X_y_train, "Sex_male")
print(f"% of men who survied: {rate_men:.2%}")

### Play ground: kernel shap
uncomment the following comments to play arround with shap.KernelExplainer to calculate the shap value estimate Shapley value

In [ ]:
# my_kernel_explainer = shap.KernelExplainer(model=model.predict_proba, data=X_train.to_numpy(), link=LINK, algorithm="kernel")

In [ ]:
# my_shap_values = my_kernel_explainer.shap_values(X_valid.iloc[[0]].to_numpy(), nsamples="auto")

### Play ground: Data profiling for exploratory data analysis (EDA)

In [ ]:
from ydata_profiling import ProfileReport

In [ ]:
# X_y_train.dtypes

In [ ]:
# profile = ProfileReport(all_X_raw_df, title="all raw data without filling the missing value report")
# profile = ProfileReport(X_y_train, title="all raw data without filling the missing value report")

In [ ]:
# profile.to_widgets()

In [ ]:
# profile.to_notebook_iframe()